In [1]:
import sympy as sp

# ── 状态向量 ─────────────────────────
px, py, pz   = sp.symbols('px py pz')          # 位置
qw, qx, qy, qz = sp.symbols('qw qx qy qz', real=True)  # 四元数 (单位化假设 |q|=1)
vx, vy, vz   = sp.symbols('vx vy vz')          # 速度
x = sp.Matrix([px, py, pz, qw, qx, qy, qz, vx, vy, vz])

# ── 控制输入 ─────────────────────────
T, wx, wy, wz = sp.symbols('T wx wy wz')       # 推力 + 机体角速度
u = sp.Matrix([T, wx, wy, wz])

g = sp.symbols('g')    # 重力常量

In [2]:
# ① 位置微分
p_dot = sp.Matrix([vx, vy, vz])

# ② 姿态微分   0.5 * q ⊗ [0, ω]
q_dot = sp.Matrix([
    0.5*(-wx*qx - wy*qy - wz*qz),
    0.5*( wx*qw + wz*qy - wy*qz),
    0.5*( wy*qw - wz*qx + wx*qz),
    0.5*( wz*qw + wy*qx - wx*qy)
])

# ③ 速度微分：R(q)·e₃ * T  - g·ẑ
e3_w = sp.Matrix([
    2*(qw*qy + qx*qz),
    2*(qy*qz - qw*qx),
    qw**2 - qx**2 - qy**2 + qz**2
])
v_dot = e3_w * T - sp.Matrix([0, 0, g])

# ④ 拼成 f(x,u)
f = sp.Matrix.vstack(p_dot, q_dot, v_dot)


In [3]:
A = f.jacobian(x)   # 10×10  (∂f/∂x)
A

Matrix([
[0, 0, 0,       0,       0,       0,       0, 1, 0, 0],
[0, 0, 0,       0,       0,       0,       0, 0, 1, 0],
[0, 0, 0,       0,       0,       0,       0, 0, 0, 1],
[0, 0, 0,       0, -0.5*wx, -0.5*wy, -0.5*wz, 0, 0, 0],
[0, 0, 0,  0.5*wx,       0,  0.5*wz, -0.5*wy, 0, 0, 0],
[0, 0, 0,  0.5*wy, -0.5*wz,       0,  0.5*wx, 0, 0, 0],
[0, 0, 0,  0.5*wz,  0.5*wy, -0.5*wx,       0, 0, 0, 0],
[0, 0, 0,  2*T*qy,  2*T*qz,  2*T*qw,  2*T*qx, 0, 0, 0],
[0, 0, 0, -2*T*qx, -2*T*qw,  2*T*qz,  2*T*qy, 0, 0, 0],
[0, 0, 0,  2*T*qw, -2*T*qx, -2*T*qy,  2*T*qz, 0, 0, 0]])

In [4]:
B = f.jacobian(u)   # 10×4   (∂f/∂u)
B

Matrix([
[                            0,       0,       0,       0],
[                            0,       0,       0,       0],
[                            0,       0,       0,       0],
[                            0, -0.5*qx, -0.5*qy, -0.5*qz],
[                            0,  0.5*qw, -0.5*qz,  0.5*qy],
[                            0,  0.5*qz,  0.5*qw, -0.5*qx],
[                            0, -0.5*qy,  0.5*qx,  0.5*qw],
[            2*qw*qy + 2*qx*qz,       0,       0,       0],
[           -2*qw*qx + 2*qy*qz,       0,       0,       0],
[qw**2 - qx**2 - qy**2 + qz**2,       0,       0,       0]])